<a href="https://colab.research.google.com/github/JunSeokCheon/kaggle_H-M_data_analysis/blob/master/del_noimage_underwear_swimwear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이미지가 없는 데이터와 underwear/swimwear류 데이터를 삭제하는 데이터 전처리를 수행합니다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [48]:
cd /content/drive/MyDrive/2nd_project_elice/inital_data

/content/drive/MyDrive/2nd_project_elice/inital_data


In [16]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
from PIL import Image
import os

In [ ]:
articles = pd.read_csv("articles.csv")
customers = pd.read_csv("customers.csv")
transactions = pd.read_csv("transactions_train.csv")

In [ ]:
print(articles.shape)
print(transactions.shape)

In [ ]:
test_articles = articles.copy()
test_transactions = transactions.copy()

In [ ]:
print(test_articles.shape)
print(test_transactions.shape)

In [7]:
ls

 age_sex_estimation.ipynb              'image&json_data'/   refine_data/
 category_data_extraction.ipynb         images/             visualization.ipynb
 del_noimage_underwear_swimwear.ipynb   inital_data/
 del_young_kid_sports.ipynb             kaggle.json


In [8]:
## root : dir과 files가 있는 path
# dirs : root 아래에 있는 폴더들
# files : root 아래에 있는 파일들

# for root, dirs, files in os.walk('./images/'):
#   print(root)
#   print(dirs)
#   print(files)

### os 모듈을 활용하여 이미지 파일에서 이미지 이름을 추출합니다.
- "이미지 이름.jpg" 형태인지 확인하고, "654847001 (1)" 같은 중복 데이터를 확인한 후 이미지이름만 리스트에 저장합니다.
- 이미지 이름에서 맨 앞에 0을 빼면 article_id와 같기 때문에 이미지 이름만 저장합니다.

In [9]:
images_names = []
for base, dirs, files in tqdm(os.walk('../images/')):
    for _files in files:
      if len(_files.split(".jpg"))==2:
        if len(_files.split(".jpg")[0].split()) == 1: # "654847001 (1)" -> 이런 파일명을 걸러주기 위해(중복 데이터 제거)
          images_names.append(_files.split(".jpg")[0].split())

0it [00:00, ?it/s]

In [10]:
len(images_names)

105100

- 이미지 이름과 article_id의 데이터프레임을 만듭니다.
- 원래 articles 데이터에 새로운 데이터 프레임을 merge하면 이미지 이름이 없는 데이터(NaN)가 이미지가 없는 데이터라고 판단할 수 있습니다.

In [11]:
image_name_df = pd.DataFrame(images_names, columns = ["image_name"])
image_name_df["article_id"] = image_name_df["image_name"].apply(lambda x: int(x[1:]))
image_name_df.head()

,image_name,article_id
0,0194242049,194242049
1,0198518010,198518010
2,0194037001,194037001
3,0192460006,192460006
4,0194270044,194270044


In [ ]:
image_article_df = test_articles[["article_id"]].merge(image_name_df, on=["article_id"], how="left")
print(image_article_df.shape)
image_article_df.head()

In [ ]:
article_no_image_df = image_article_df.loc[image_article_df.image_name.isna()]
print(article_no_image_df.shape)
article_no_image_df.head()

In [ ]:
test_articles.head()

- product_group_name이 underwear인 데이터와 swimwear 데이터의 article_id를 리스트로 만들어서 저장합니다.
- 이미지가 없는 데이터의 article_id를 리스트로 만들어서 저장합니다.

In [ ]:
article_underwear_del = test_articles[test_articles["product_group_name"] == "Underwear"]['article_id'].tolist()

In [ ]:
article_swimwear_del = test_articles[test_articles["product_group_name"] == "Swimwear"]['article_id'].tolist()

In [ ]:
no_image_articleid = article_no_image_df['article_id'].tolist()

- articles.csv에서 이미지가 없는 데이터와 underwear 데이터를 삭제합니다.

In [ ]:
for id in no_image_articleid:
  index1 = test_articles[test_articles['article_id'] == id].index
  test_articles = test_articles.drop(index = index1, axis = 0)

In [ ]:
for id in article_underwear_del:
  index1 = test_articles[test_articles['article_id'] == id].index
  test_articles = test_articles.drop(index = index1, axis = 0)

In [ ]:
for id in article_swimwear_del:
  index1 = test_articles[test_articles['article_id'] == id].index
  test_articles = test_articles.drop(index = index1, axis = 0)

In [ ]:
# 4~6월 거래내역 데이터만 사용(전체 데이터 2천만개라서 처리에 곤란)
test_transactions["t_dat"] = pd.to_datetime(test_transactions['t_dat'])
test_transactions = test_transactions.loc[test_transactions.t_dat >= pd.to_datetime('2020-04-01')] #
test_transactions = test_transactions.loc[test_transactions.t_dat <= pd.to_datetime('2020-06-01')] #

- transactions.csv에서 이미지가 없는 데이터와 underwear 데이터를 삭제합니다.
- customers.csv는 article에 대한 정보가 없기 때문에 제외해도 됩니다.

In [ ]:
for id in no_image_articleid:
  index1 = test_transactions[test_transactions['article_id'] == id].index
  test_transactions = test_transactions.drop(index = index1, axis = 0)

In [ ]:
for id in article_underwear_del:
  index1 = test_transactions[test_transactions['article_id'] == id].index
  test_transactions = test_transactions.drop(index = index1, axis = 0)

In [ ]:
for id in article_swimwear_del:
  index1 = test_transactions[test_transactions['article_id'] == id].index
  test_transactions = test_transactions.drop(index = index1, axis = 0)

In [ ]:
# 이미지 없는 것과 속옷류 데이터 삭제(2020-04 - 2020-06 데이터에서)
# article : 105542 -> 99638
# transaction : 31788324 -> 2491536
print(test_articles.shape)
print(test_transactions.shape)

(99638, 25)
(2491536, 5)


In [ ]:
test_articles = test_articles.reset_index(drop=True)

In [ ]:
test_articles

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,111565001,111565,20 den 1p Stockings,304,Underwear Tights,Socks & Tights,1010016,Solid,9,Black,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,"Semi shiny nylon stockings with a wide, reinfo..."
4,111565003,111565,20 den 1p Stockings,302,Socks,Socks & Tights,1010016,Solid,13,Beige,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,"Semi shiny nylon stockings with a wide, reinfo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99633,953450001,953450,5pk regular Placement1,302,Socks,Socks & Tights,1010014,Placement print,9,Black,...,Socks Bin,F,Menswear,3,Menswear,26,Men Underwear,1021,Socks and Tights,Socks in a fine-knit cotton blend with a small...
99634,953763001,953763,SPORT Malaga tank,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey,A,Ladieswear,1,Ladieswear,2,H&M+,1005,Jersey Fancy,Loose-fitting sports vest top in ribbed fast-d...
99635,956217002,956217,Cartwheel dress,265,Dress,Garment Full body,1010016,Solid,9,Black,...,Jersey,A,Ladieswear,1,Ladieswear,18,Womens Trend,1005,Jersey Fancy,"Short, A-line dress in jersey with a round nec..."
99636,957375001,957375,CLAIRE HAIR CLAW,72,Hair clip,Accessories,1010016,Solid,9,Black,...,Small Accessories,D,Divided,2,Divided,52,Divided Accessories,1019,Accessories,Large plastic hair claw.


In [ ]:
test_transactions = test_transactions.reset_index(drop=True)

In [ ]:
test_transactions

,t_dat,customer_id,article_id,price,sales_channel_id
0,2020-04-01,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,727808001,0.067780,2
1,2020-04-01,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,727808007,0.067780,2
2,2020-04-01,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,783504004,0.019051,2
3,2020-04-01,000eae69313b4fc1824fa7e439f168cc140bf4c3f3a7e9...,599580061,0.012034,2
4,2020-04-01,000eae69313b4fc1824fa7e439f168cc140bf4c3f3a7e9...,844294002,0.019576,2
...,...,...,...,...,...
2491531,2020-06-01,fffb04448b4082028fcf3f205b05f0ee846da598eeca1f...,873977001,0.050831,2
2491532,2020-06-01,fffb04448b4082028fcf3f205b05f0ee846da598eeca1f...,851317001,0.042356,2
2491533,2020-06-01,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,698286007,0.025407,2
2491534,2020-06-01,fffef3b6b73545df065b521e19f64bf6fe93bfd450ab20...,892051002,0.028797,2


In [ ]:
ls

 age_sex_estimation.ipynb        'image&json_data'/   refine_data/
 category_data_extraction.ipynb   images/             visualization.ipynb
 data_preprocessing.ipynb         inital_data/
 del_young_kid.ipynb              kaggle.json


In [ ]:
# test_articles.to_csv('./refine_data/articles.csv')
# test_transactions.to_csv('./refine_data/transactions.csv')

### 여성 속옷 데이터가 underwear 밖에 없는 줄 알고 underwear 데이터만 삭제했지만, 뒤늦게 swimwear 제품도 있다는 것을 알고 기존 정제된 데이터에서 underwear/Unknown 제품을 추가로 삭제하는 작업입니다.

In [18]:
cd /content/drive/MyDrive/2nd_project_elice/refine_data

/content/drive/MyDrive/2nd_project_elice/refine_data


In [24]:
articles = pd.read_csv("articles.csv")
customers = pd.read_csv("customers.csv")
transactions = pd.read_csv("transactions.csv")

In [25]:
test_articles = articles.copy()
test_transactions = transactions.copy()

In [32]:
article_swimwear_del = test_articles[(test_articles["product_group_name"] == "Swimwear") | (test_articles["product_group_name"] == "Unknown") | (test_articles["product_group_name"] == "Underwear/nightwear")]['article_id'].tolist()

In [33]:
for id in article_swimwear_del:
  index1 = test_articles[test_articles['article_id'] == id].index
  test_articles = test_articles.drop(index = index1, axis = 0)

In [36]:
# 4~6월 거래내역 데이터만 사용(전체 데이터 2천만개라서 처리에 곤란)
test_transactions["t_dat"] = pd.to_datetime(test_transactions['t_dat'])
test_transactions = test_transactions.loc[test_transactions.t_dat >= pd.to_datetime('2020-04-01')] #
test_transactions = test_transactions.loc[test_transactions.t_dat <= pd.to_datetime('2020-06-01')] #

In [37]:
for id in article_swimwear_del:
  index1 = test_transactions[test_transactions['article_id'] == id].index
  test_transactions = test_transactions.drop(index = index1, axis = 0)

In [38]:
test_articles = test_articles.reset_index(drop=True)

In [39]:
test_articles.head()

,Unnamed: 0,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,3,111565001,111565,20 den 1p Stockings,304,Underwear Tights,Socks & Tights,1010016,Solid,9,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,"Semi shiny nylon stockings with a wide, reinfo..."
4,4,111565003,111565,20 den 1p Stockings,302,Socks,Socks & Tights,1010016,Solid,13,...,Tights basic,B,Lingeries/Tights,1,Ladieswear,62,"Womens Nightwear, Socks & Tigh",1021,Socks and Tights,"Semi shiny nylon stockings with a wide, reinfo..."


In [45]:
test_articles[(test_articles["product_group_name"] == "Swimwear") | (test_articles["product_group_name"] == "Unknown") | (test_articles["product_group_name"] == "Underwear/nightwear")]

,Unnamed: 0,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc


In [40]:
test_transactions = test_transactions.reset_index(drop=True)

In [41]:
test_transactions.head()

,Unnamed: 0,t_dat,customer_id,article_id,price,sales_channel_id
0,0,2020-04-01,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,727808001,0.067780,2
1,1,2020-04-01,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,727808007,0.067780,2
2,2,2020-04-01,00083cda041544b2fbb0e0d2905ad17da7cf1007526fb4...,783504004,0.019051,2
3,3,2020-04-01,000eae69313b4fc1824fa7e439f168cc140bf4c3f3a7e9...,599580061,0.012034,2
4,4,2020-04-01,000eae69313b4fc1824fa7e439f168cc140bf4c3f3a7e9...,844294002,0.019576,2


In [49]:
test_articles.to_csv('../refine_data/articles.csv')
test_transactions.to_csv('../refine_data/transactions.csv')